# Introduzione
Utilizzare l'environment conda **whisper**.

Questo script serve ad trascrivere  i file audio delle puntate di camera cafe e salvarli in file txt. 

# Importazione pacchetti

In [1]:
import os # lavorare con directory...

from tqdm import tqdm # Barre di progresso

import whisper # Importare il modello di OPENAI adibito alla trascrizione di audio

import warnings # per gestire i warning
# Rimuovere questo warning, evita che  la barra di progesso tqdm venga interrotta continuamente diventando inutile
warnings.filterwarnings("ignore", message="FP16 is not supported on CPU; using FP32 instead")


# Importare file audio da trascrivere

è stata aggiunta una parte che guarda ai file già trascritti e fa in modo da cancellarli dalla coda di quelli da estrarre.    
In questo modo il procedimento di trascrizione può essere interrotto e iniziato di nuovo senza problemi.

In [2]:
# Path per la directory nella quale ci sono i file audio delle varie stagioni
path = "/Users/mattia/Desktop/Università/Data Science in Python/13) Neural Networks/Data/Audio/Camera Cafe"
path_esportazione = "/Users/mattia/Desktop/Università/Data Science in Python/13) Neural Networks/Data/Trascrizioni Audio/Camera Cafe"

# Nome delle cartelle contenente i file audio per ogni stagione
stagioni = {#1:'Camera Cafe Prima Stagione',
            #2:"Camera Cafe Seconda Stagione",
            #3:"Camera Cafe Terza Stagione",
            #4:"Camera Cafe Quarta Stagione"#,
            5:"Camera Cafe Quinta Stagione" # per ora non serve
            }



# Inizializzare un dizionario. In questo dizionario ogni stagione sarà una chiave ed ad essa saranno associati
# tutti i nomi dei file audio appartenenti a quella stagione
puntate = {}

# Parametri di controllo per capire il numero di file audio in totale e per ogni stagione
numero_puntate = 0   # per contare file audio totali
numero_stagione = {} # per contare file audio per stagione

# Iterare sulle stagioni
for key in stagioni.keys():

   # Muoversi alla directory della k-esima stagione degli già estratti
   os.chdir(f"{path_esportazione}/{stagioni[key]}")

   # Prendere tutti i file nella directory nella k-esima stagione già estratti, cambiargli l'estensione
   # solo per facilitare le operazioni di matching
   files_gia_estratti =  [f"{stagioni[key]}/{file}".replace(".txt",".wav") for file in os.listdir() if "txt" in file]


   # Muoversi alla directory della k-esima stagione
   os.chdir(f"{path}/{stagioni[key]}")


   files = [f"{stagioni[key]}/{file}" for file in os.listdir() if "wav" in file] #QUI ANDRà CAMBIATO SE L'ESTENSIONE DEI FILE AUDIO
   # è DIVERSA DA wav, magari mp4
   files = [file for file in files if file not in files_gia_estratti]

   numero_puntate += len(files)                    # aggiungere il numero di episodi per la k-esima stagione
   numero_stagione[f"Stagione {key}"] = len(files) # salvare il numero di episodi per la k-esima stagione
   
   # Salvare il nome dei file audio per ogni stagione
   puntate[key] = files

print(f"Ci sono file audio riguardanti {numero_puntate} puntate\n")
print(f"Numero di puntate da estrarre per ogni stagione:\n  {numero_stagione}")

Ci sono file audio riguardanti 3 puntate

Numero di puntate da estrarre per ogni stagione:
  {'Stagione 5': 3}


# Processo di trascrizione

Scelto il modello turbo perchè rispetto a il large sembra sbagliare le congiunzioni e le piccole parole.       
Siccome congiunzioni e parole di uso comune come intercalare e altro vengono poi eliminate dallo stemming il   
miglior compromesso è il modello **turbo** in quanto la qualità è quasi buona quanto il large ma è molto più veloce.   

In [3]:
# Definire il  modello whisper da utilizzare e selezionarne la versione
model = whisper.load_model("turbo") # Caricare il modello 

In [4]:
# Iterare sulla stagione
for key in puntate.keys():
   print(f"Estrazione stagione {key}")
   # Estrarre tutte le puntate per la k-esima stagione
   puntate_stagione_k_esima = puntate[key]
   for i in tqdm(range(len(puntate_stagione_k_esima))):

      # Prendere il percorso file parziale (directory stagione + nome puntata) della k-esima puntata 
      puntata = puntate_stagione_k_esima[i]

      # Creare il path per il file audio
      file_audio = f"{path}/{puntata}"

      # Runnare il modello e trasformare l'audio in testo
      result = model.transcribe(file_audio,language="it")

      
      nome_puntata = puntata.replace(".wav","")

      with open(f"{path_esportazione}/{nome_puntata}.txt","w+") as trascrizione :
         trascrizione.write(result["text"])


Estrazione stagione 5


 33%|███▎      | 1/3 [05:44<11:28, 344.43s/it]


KeyboardInterrupt: 